In [1]:
!sudo pip install -q transformers --upgrade

In [2]:
from transformers import AutoModelForCausalLM

In [3]:
import torch

In [4]:
model = AutoModelForCausalLM.from_pretrained("/data/mistral/query-to-mql/exp-10/nov-20/merged-model",
                                            torch_dtype=torch.float16, device_map="auto")


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
model.to_bettertransformer()